## Run this cell to connect to your GIS and get started:

In [ ]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
# @Title: Portal Cleanup Notebook
#
# @Purpose: this notebook is a sample of how the ArcGIS Python
#    API can be used to adminster a portal. Specifically, this
#    notebook automates the process of finding items tagged as
#    TEST and by finding Inactive Users
#
# @Creator: phornstein@esri.com
# @Date: March 2021
#
# @Versions: ArcGIS Python API v1.8.4
#
# @License: See end of notebook
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

import time
import requests
import json
import warnings
from datetime import datetime as dt
from datetime import timedelta
from arcgis.gis import GIS, UserManager
import pandas as pd

gis = GIS("home")

## Get Test Items

In [ ]:
'''
Search the entire portal for any item with the word
# test in the name or tags
#
# For each "test item" found, check to only get items 
# with test as a tag and then add the item, tags, 
# user, and user email to a list for processing
'''
tag_to_delete = 'test' #the tag to search for and delete, this can be anything

def getItemsByTag(tag_to_search_for):
    '''
    Searchs Portal for all items with a specific tag
    
    Parameters
    ----------
    tag_to_delete : str
        The tag to search for
    
    Returns
    -------
    Pandas data frame with information on each item with a matching tag
    '''
    results = gis.content.search(tag_to_search_for)
    print('Items matching \'{}\''.format(tag_to_search_for))
    print('_'*125)
    items = [print("ITEM: {0:50} | {1:25} | OWNER: {2:30}|".format(r.title[:50],r.type[:25],r.owner)) for r in results]
    print('_'*125)

    items_to_delete = []
    for item in results:
        for tag in item.tags:
            if 'test' == tag.lower():
                usr = gis.users.search(query='username:{}'.format(item.owner))[0]
                report = {'item':item.title,
                         'name':usr.fullName,
                         'tags':', '.join(item.tags),
                         'email':usr.email}
                items_to_delete.append(report)
    items_df = pd.DataFrame.from_dict(items_to_delete)

    print('\n\nItems tagged with \'{}\''.format(tag_to_search_for))
    print('_'*125)
    items = [print("ITEM: {0:50} | TAGS: {1:25} | OWNER: {2:25}|".format(i['item'][:50],i['tags'][:25],i['name'][:25],)) for i in items_to_delete]
    print('_'*125)
    return items_df

items_df = getItemsByTag(tag_to_delete)

## Get Inactive Users

In [ ]:
'''
# Search portal using UserManager to get all users
#
# Use datetime.timedelta to check the last log in of each
# portal user and see if it longer than the tlate time. 
# tlate being the maximum number of days without login before
# a user is considered inactive.
#
# Add all inactive users to a list for with necessary contact info
'''
tlate_days = 31 #the number of days since last login before notification

def getInactiveUsers(tlate_days):
    '''
    Find users who have not logged in to the Portal in a defined time
    
    Parameters
    ----------
    tlate_days : int
        Number of days without activity to be considered inactive
    
    Returns
    -------
    Pandas data frame with information on each inactive user
    '''
    user_manager = UserManager(gis)
    num_users = user_manager.counts('user_type', as_df=False)[0]['count']
    users = gis.users.search(query=None, max_users=num_users)
    today = dt.now()
    tlate_time = today - timedelta(days = tlate_days)

    tlate_users = []
    for user in users:
        last_login = dt.fromtimestamp(int(user.lastLogin/1000))
        if last_login:
            login_delta = today - last_login
            if login_delta.days > tlate_days:
                report = {'name':user.fullName,
                          'username':user.username,
                         'tlate':login_delta.days,
                         'lastLogin':dt.strftime(last_login,'%Y-%m-%d %H:%M:%S'),
                         'email':user.email,
                         'userLevel':user.level,
                         'userRole':user.role}
                tlate_users.append(report)

    return pd.DataFrame.from_dict(tlate_users)

users_df = getInactiveUsers(tlate_days)
users_df

## Send Notifications

In [ ]:
'''
# Rest URL to Geoevent server which will broker email notifications
# For the purpose of this sample a geoevent server is used to 
# automate output emails. A python SMTP package can be used in exchange
'''
geoevent_url = '<GEOEVENT REST URL'

In [ ]:
def postToGeoevent(jsonData,geoeventRestURL):
    '''
    Posts JSON payload to Geoevent Rest connector
    
    Parameters
    ----------
    jsonData : dict
        The formated data to be sent to geoevent. Dictionary keys
        should match the input geoevent definition
        
    geoeventRestURL : str
        A string of the Geoevent Rest URL the data will be published to
    '''
    warnings.filterwarnings("ignore") #ignore ssl cert warnings 
    headers = {'Content-type':'application/json', 'Accept':'text/plain'} #standard post headers for geoevent
    try:
        response = requests.post(geoeventRestURL, data=json.dumps(jsonData),headers=headers, verify=False)
    except:
        print('Failed',jsonData)

In [ ]:
'''
# Create Item deletion Alerts
'''

def generateDeleteMessage(row):
    message = \
    '''{},
    
    The item below will be deleted from the Defense Portal in 24 hours.
    
    Reason: tagged with \'test\'
    Item: {}
    Item Tags: {}
    
    To prevent this item from being deleted please change the tags.
    '''.format(row['name'],row['item'],row['tags'])
    output = {'email':row['email'],
             'subject':'ACTION: Items to be deleted',
             'message':message}
    postToGeoevent(output,geoevent_url)

    
delete_df= items_df.apply(generateDeleteMessage,axis=1)
print('{} deletion emails sent.'.format(len(delete_df.index)))

In [ ]:
'''
# Create inactive user Alerts
'''
def generateInactiveUserMessage(row):
    message = \
    '''{},
    
    Our records show that you have not logged into the GC Defense Portal in {} days. 
    Your last login was on {}.
    You are currently assigned the permissions -> User Level: {} | User Role: {}
    
    If you do not log in within the next 24 hours your persission will be set to User Level: 1 | User Role: Viewer
    
    If you have not logged into the Portal in over 60 days your account will be set to inactive.
    
    Portal URL: https://defense.esri.com/portal/home/
    '''.format(row['name'],row['tlate'],row['lastLogin'],row['userLevel'],row['userRole'])
    output = {'subject':'ACTION: Inactive User Alert',
             'email':row['email'],
             'message':message}
    postToGeoevent(output,geoevent_url)
    
inactive_users_df = users_df.apply(generateInactiveUserMessage,axis=1)
print('{} inactive user emails sent.'.format(len(inactive_users_df.index)))

In [ ]:
'''
# Wait 24 hours before proceeding
'''
time.sleep(86400) #wait 24 hours before executing deletions

## Run Deletions

In [ ]:
'''
# Check the portal again to get items still tagged with test
'''
items_df_2 = getItemsByTag(tag_to_delete)

In [ ]:
'''
# Use pandas merge inner join to find the differnce of the 
# items to delete from 24 hours prior and current time
'''
items_merge = items_df.merge(items_df_2,how='inner',left_on='item',right_on='item')
items_merge

In [ ]:
'''
# Delete all items tagged with test. Then run search again
# to confirm all items have been removed
'''
items_not_deleted = []
def deleteItem(row):
    try:
        items = gis.content.search(row['item'])
        for item in items:
            item.delete(force=True)
    except Exception as e:
        items_not_deleted.append('ITEM: {} | OWNER: {} | {}'.format(row['item'],row['name_x'],e))
        print('ITEM: {} | OWNER: {} | {}'.format(row['item'],row['name_x'],e))
        
items_merge.apply(deleteItem,axis=1)
remaining_items_to_delete = getItemsByTag(tag_to_delete)

## Downgrade Inactive Users

In [ ]:
'''
# Check for inactive users again to look for change in activity
'''
users_df_2 = getInactiveUsers(tlate_days)
users_df_2

In [ ]:
'''
# Use pandas merge inner join to find the differnce of the 
# items to delete from 24 hours prior and current time
'''
users_merge = users_df.merge(users_df_2,how='inner',left_on='name',right_on='name')
users_merge

In [ ]:
'''
# Update or disable all tlate users. Check for inactive
# users again to confirm all have been removed
'''
users_not_downgraded = []
def updateInactiveUsers(row):
    try:
        usr = gis.users.search(query='username:{}'.format(row['username_x']))[0]
        usr.update_license_type('Viewer')
        usr.update_level(1)
        if row['tlate_x'] > 60:
            usr.disable()
    except Exception as e:
        users_not_downgraded.append('USER: {} | INACTIVE DAYS: {} | {}'.format(row['username_x'],row['tlate_x'],e))
        
users_merge = users_merge.apply(updateInactiveUsers,axis=1)
remaining_inactive_users = getInactiveUsers(tlate_days)
remaining_inactive_users

## Generate Admin Report

In [ ]:
'''
# Create Admin report
'''
admin_emails = [('NAME','ADMIN EMAIL')] #list of tuples of admins

count_items = len(items_merge.index) #the number of items deleted
error_items = '\n'.join(items_not_deleted) #all items that could not be deleted

users = ['USER: {} | INACTIVE DAYS: {}'.format(row[1]['username_x'],row[1]['tlate_x']) for row in users_merge.iterrows()]
inactive_users = '\n'.join(users)
error_users = '\n'.join(users_not_downgraded)

for admin in admin_emails:
    message = \
    '''{},

Below is your weekly Portal Admin report:

Items deleted tagged with test: {}
_______________________________________________________________

There were errors deleting the following items:

{}\n
_______________________________________________________________

The following users were downgraded or disabled for inactivity:

{}\n
_______________________________________________________________

There were errors processing the following inactive users:

{}\n
_______________________________________________________________

Stay classy, Honeybadger!'''.format(admin[0],count_items,error_items,inactive_users,error_users)

    data = {'subject':'Portal weekly admin report',
           'email':admin[1],
           'message':message}
    postToGeoevent(data,geoevent_url)
print('{} emails sent.'.format(len(admin_emails)))

## License